In [ ]:
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from ml_wireless_classification.base.SignalUtils import (
    compute_instantaneous_features, compute_modulation_index, compute_spectral_asymmetry,
    instantaneous_frequency_deviation, spectral_entropy, envelope_mean_variance,
    spectral_flatness, spectral_peaks_bandwidth, zero_crossing_rate, compute_fft_features,
    autocorrelation, is_digital_signal, compute_kurtosis, compute_skewness,
    compute_spectral_energy_concentration, compute_instantaneous_frequency_jitter,
    compute_spectral_kurtosis, compute_higher_order_cumulants, compute_crest_factor,
    compute_spectral_entropy, compute_energy_spread, compute_autocorrelation_decay,
    compute_rms_of_instantaneous_frequency, compute_entropy_of_instantaneous_frequency,
    compute_envelope_variance, compute_papr
)

# Global dictionary to store feature names and values
feature_dict = {}

def add_feature(name, func, *args):
    """Try to add a feature by checking the shape and ensuring it’s a scalar."""
    try:
        value = func(*args)
        # If value is an array, check if it is scalar (single value)
        if np.isscalar(value) or (isinstance(value, np.ndarray) and value.size == 1):
            feature_dict[name] = value.item() if isinstance(value, np.ndarray) else value
        else:
            print(f"Warning: Feature '{name}' has incorrect shape and was not added.")
    except Exception as e:
        print(f"Error computing feature '{name}': {e}")

def extract_features(data):
    features = []
    labels = []
    snrs = []

    for key, signals in data.items():
        mod_type, snr = key
        for signal in signals:
            real_part, imag_part = signal[0], signal[1]
            complex_signal = real_part + 1j * imag_part

            # Reset feature dictionary for each signal
            global feature_dict
            feature_dict = {}

            # Add features with validation
            add_feature("Inst. Freq. Dev", instantaneous_frequency_deviation, complex_signal)
            add_feature("Spectral Entropy", spectral_entropy, real_part)
            add_feature("Envelope Mean", lambda x: envelope_mean_variance(x)[0], real_part)
            add_feature("Envelope Variance", lambda x: envelope_mean_variance(x)[1], real_part)
            add_feature("Spectral Flatness", spectral_flatness, real_part)
            add_feature("Spectral Peaks", lambda x: spectral_peaks_bandwidth(x)[0], real_part)
            add_feature("Bandwidth", lambda x: spectral_peaks_bandwidth(x)[1], real_part)
            add_feature("Zero Crossing Rate", zero_crossing_rate, real_part)
            add_feature("Amplitude Mean", lambda x: np.mean(compute_instantaneous_features(x)[0]), real_part)
            add_feature("Phase Variance", lambda x: np.var(compute_instantaneous_features(x)[1]), real_part)
            add_feature("Modulation Index", compute_modulation_index, real_part)
            add_feature("Spectral Sparsity", compute_spectral_asymmetry, real_part)
            add_feature("Envelope Ratio", lambda x: envelope_mean_variance(x)[0] / (envelope_mean_variance(x)[1] + 1e-10), real_part)
            add_feature("FFT Center Freq", lambda x: compute_fft_features(x)[0], real_part)
            add_feature("FFT Peak Power", lambda x: compute_fft_features(x)[1], real_part)
            add_feature("FFT Avg Power", lambda x: compute_fft_features(x)[2], real_part)
            add_feature("FFT Std Dev Power", lambda x: compute_fft_features(x)[3], real_part)
            add_feature("Kurtosis", compute_kurtosis, real_part)
            add_feature("Skewness", compute_skewness, real_part)
            add_feature("HOC-2", lambda x: compute_higher_order_cumulants(x, order=2), real_part)
            add_feature("HOC-3", lambda x: compute_higher_order_cumulants(x, order=3), real_part)
            add_feature("HOC-4", lambda x: compute_higher_order_cumulants(x, order=4), real_part)
            add_feature("Crest Factor", compute_crest_factor, real_part)
            add_feature("Spectral Entropy Value", compute_spectral_entropy, real_part)
            add_feature("Autocorr Decay", compute_autocorrelation_decay, real_part)
            add_feature("RMS Instant Freq", compute_rms_of_instantaneous_frequency, real_part)
            add_feature("Entropy Instant Freq", compute_entropy_of_instantaneous_frequency, real_part)
            add_feature("Envelope Variance", compute_envelope_variance, real_part)
            add_feature("PAPR", compute_papr, real_part)

            # Add SNR as a feature
            feature_dict["SNR"] = snr  # Include SNR as part of the features

            # Append the feature values and label
            features.append(list(feature_dict.values()))
            labels.append(mod_type)

    return np.array(features), labels

# Load the RML2016.10a_dict.pkl file with explicit encoding
with open("../RML2016.10a_dict.pkl", "rb") as f:
    data = pickle.load(f, encoding="latin1")

# Feature extraction for all signals
features, labels = extract_features(data)

# Encode labels for classification
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.3, random_state=42)

# Train a single classifier on the entire dataset
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate accuracy for each SNR level
unique_snrs = sorted(set(X_test[:, -1]))  # Get unique SNR levels from test set
accuracy_per_snr = []

for snr in unique_snrs:
    # Select samples with the current SNR
    snr_indices = np.where(X_test[:, -1] == snr)
    X_snr = X_test[snr_indices]
    y_snr = y_test[snr_indices]

    # Predict and calculate accuracy
    y_pred = clf.predict(X_snr)
    accuracy = accuracy_score(y_snr, y_pred)
    accuracy_per_snr.append(accuracy * 100)  # Convert to percentage

    print(f"SNR: {snr} dB, Accuracy: {accuracy * 100:.2f}%")

# Plot Recognition Accuracy vs. SNR
plt.figure(figsize=(10, 6))
plt.plot(unique_snrs, accuracy_per_snr, 'b-o', label='Recognition Accuracy')
plt.xlabel("SNR (dB)")
plt.ylabel("Recognition Accuracy (%)")
plt.title("Recognition Accuracy vs. SNR")
plt.legend()
plt.grid(True)
plt.ylim(0, 100)
plt.show()

# Feature importance for the classifier
feature_names = list(feature_dict.keys())
importances = clf.feature_importances_
plt.figure(figsize=(10, 8))
plt.barh(feature_names, importances, color='skyblue')
plt.xlabel("Feature Importance")
plt.title("Feature Importance for Classification")
plt.show()

# Confusion matrix for overall test set
y_pred_test = clf.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Modulation Classification")
plt.show()



In [ ]:
# denoise and filter with a second method

from scipy.signal import butter, sosfilt
import pywt

def apply_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    sos = butter(order, [lowcut, highcut], btype='band', fs=fs, output='sos')
    return sosfilt(sos, signal)

def wavelet_denoising(signal, wavelet='db1', level=2):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    thresholded_coeffs = [pywt.threshold(c, np.std(c), mode='soft') for c in coeffs]
    return pywt.waverec(thresholded_coeffs, wavelet)

def extract_features_with_filtering(data, fs=2.5, lowcut=0.1, highcut=2.5):
    features = []
    labels = []

    for key, signals in data.items():
        mod_type, snr = key
        for signal in signals:
            real_part, imag_part = signal[0], signal[1]
            
            # Apply Bandpass Filter
            filtered_real = apply_bandpass_filter(real_part, lowcut, highcut, fs)
            filtered_imag = apply_bandpass_filter(imag_part, lowcut, highcut, fs)

            # Apply Wavelet Denoising
            real_denoised = wavelet_denoising(filtered_real)
            imag_denoised = wavelet_denoising(filtered_imag)
            complex_denoised_signal = real_denoised + 1j * imag_denoised

            # Reset feature dictionary for each signal
            global feature_dict
            feature_dict = {}

            # Add features with validation
            add_feature("Inst. Freq. Dev", instantaneous_frequency_deviation, complex_denoised_signal)
            add_feature("Spectral Entropy", spectral_entropy, real_denoised)
            add_feature("Envelope Mean", lambda x: envelope_mean_variance(x)[0], real_denoised)
            add_feature("Envelope Variance", lambda x: envelope_mean_variance(x)[1], real_denoised)
            add_feature("Spectral Flatness", spectral_flatness, real_denoised)
            add_feature("Spectral Peaks", lambda x: spectral_peaks_bandwidth(x)[0], real_denoised)
            add_feature("Bandwidth", lambda x: spectral_peaks_bandwidth(x)[1], real_denoised)
            add_feature("Zero Crossing Rate", zero_crossing_rate, real_denoised)
            add_feature("Amplitude Mean", lambda x: np.mean(compute_instantaneous_features(x)[0]), real_denoised)
            add_feature("Phase Variance", lambda x: np.var(compute_instantaneous_features(x)[1]), real_denoised)
            add_feature("Modulation Index", compute_modulation_index, real_denoised)
            add_feature("Spectral Sparsity", compute_spectral_asymmetry, real_denoised)
            add_feature("Envelope Ratio", lambda x: envelope_mean_variance(x)[0] / (envelope_mean_variance(x)[1] + 1e-10), real_denoised)
            add_feature("FFT Center Freq", lambda x: compute_fft_features(x)[0], real_denoised)
            add_feature("FFT Peak Power", lambda x: compute_fft_features(x)[1], real_denoised)
            add_feature("FFT Avg Power", lambda x: compute_fft_features(x)[2], real_denoised)
            add_feature("FFT Std Dev Power", lambda x: compute_fft_features(x)[3], real_denoised)
            add_feature("Kurtosis", compute_kurtosis, real_denoised)
            add_feature("Skewness", compute_skewness, real_denoised)
            add_feature("HOC-2", lambda x: compute_higher_order_cumulants(x, order=2), real_denoised)
            add_feature("HOC-3", lambda x: compute_higher_order_cumulants(x, order=3), real_denoised)
            add_feature("HOC-4", lambda x: compute_higher_order_cumulants(x, order=4), real_denoised)
            add_feature("Crest Factor", compute_crest_factor, real_denoised)
            add_feature("Spectral Entropy Value", compute_spectral_entropy, real_denoised)
            add_feature("Autocorr Decay", compute_autocorrelation_decay, real_denoised)
            add_feature("RMS Instant Freq", compute_rms_of_instantaneous_frequency, real_denoised)
            add_feature("Entropy Instant Freq", compute_entropy_of_instantaneous_frequency, real_denoised)
            add_feature("Envelope Variance", compute_envelope_variance, real_denoised)
            add_feature("PAPR", compute_papr, real_denoised)

            # Add SNR as a feature
            feature_dict["SNR"] = snr  # Include SNR as part of the features

            # Append the feature values and label
            features.append(list(feature_dict.values()))
            labels.append(mod_type)

    return np.array(features), labels

# Apply the new feature extraction with filtering
filtered_features, filtered_labels = extract_features_with_filtering(data)

# Encode labels for classification
filtered_encoded_labels = label_encoder.fit_transform(filtered_labels)

# Split dataset into training and testing sets
X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = train_test_split(
    filtered_features, filtered_encoded_labels, test_size=0.3, random_state=42
)

# Train a classifier on the filtered data
clf_filtered = RandomForestClassifier(n_estimators=100, random_state=42)
clf_filtered.fit(X_train_filtered, y_train_filtered)

# Evaluate accuracy for each SNR level with filtered data
accuracy_per_snr_filtered = []
for snr in unique_snrs:
    snr_indices_filtered = np.where(X_test_filtered[:, -1] == snr)
    X_snr_filtered = X_test_filtered[snr_indices_filtered]
    y_snr_filtered = y_test_filtered[snr_indices_filtered]

    # Predict and calculate accuracy
    y_pred_filtered = clf_filtered.predict(X_snr_filtered)
    accuracy = accuracy_score(y_snr_filtered, y_pred_filtered)
    accuracy_per_snr_filtered.append(accuracy * 100)

    print(f"SNR: {snr} dB, Accuracy (filtered): {accuracy * 100:.2f}%")

# Plot Comparison of Recognition Accuracy vs. SNR
plt.figure(figsize=(10, 6))
plt.plot(unique_snrs, accuracy_per_snr, 'b-o', label='Original Recognition Accuracy')
plt.plot(unique_snrs, accuracy_per_snr_filtered, 'g-o', label='Filtered Recognition Accuracy')
plt.xlabel("SNR (dB)")
plt.ylabel("Recognition Accuracy (%)")
plt.title("Recognition Accuracy vs. SNR (Original vs. Filtered)")
plt.legend()
plt.grid(True)
plt.ylim(0, 100)
plt.show()

# Feature importance for the filtered classifier
importances_filtered = clf_filtered.feature_importances_
plt.figure(figsize=(10, 8))
plt.barh(feature_names, importances_filtered, color='skyblue')
plt.xlabel("Feature Importance")
plt.title("Feature Importance for Filtered Classification")
plt.show()

# Confusion matrix for filtered test set
y_pred_test_filtered = clf_filtered.predict(X_test_filtered)
conf_matrix_filtered = confusion_matrix(y_test_filtered, y_pred_test_filtered)
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix_filtered, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Filtered Modulation Classification")
plt.show()

# Print Classification Report for filtered data
print("Classification Report for Filtered Modulation Types:")
print(classification_report(y_test_filtered, y_pred_test_filtered, target_names=label_encoder.classes_))


In [ ]:
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter, sosfilt, medfilt
import pywt
from ml_wireless_classification.base.SignalUtils import (
    compute_instantaneous_features, compute_modulation_index, compute_spectral_asymmetry,
    instantaneous_frequency_deviation, spectral_entropy, envelope_mean_variance,
    spectral_flatness, spectral_peaks_bandwidth, zero_crossing_rate, compute_fft_features,
    autocorrelation, is_digital_signal, compute_kurtosis, compute_skewness,
    compute_spectral_energy_concentration, compute_instantaneous_frequency_jitter,
    compute_spectral_kurtosis, compute_higher_order_cumulants, compute_crest_factor,
    compute_spectral_entropy, compute_energy_spread, compute_autocorrelation_decay,
    compute_rms_of_instantaneous_frequency, compute_entropy_of_instantaneous_frequency,
    compute_envelope_variance, compute_papr
)

# Filtering functions
def apply_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    sos = butter(order, [lowcut, highcut], btype='band', fs=fs, output='sos')
    return sosfilt(sos, signal)

def apply_median_filter(signal, kernel_size=5):
    return medfilt(signal, kernel_size=kernel_size)

def apply_wavelet_denoising(signal, wavelet="db1", level=1):
    coeffs = pywt.wavedec(signal, wavelet, mode="soft")
    threshold = np.median(np.abs(coeffs[-level])) / 0.6745
    denoised_coeffs = [pywt.threshold(c, threshold, mode="soft") for c in coeffs]
    return pywt.waverec(denoised_coeffs, wavelet)

# Combined filtering function
def preprocess_signal(signal, fs, lowcut=0.5, highcut=40, kernel_size=5, wavelet="db1", level=1):
    # Step 1: Bandpass filter
    filtered_signal = apply_bandpass_filter(signal, lowcut, highcut, fs)
    # Step 2: Median filter
    median_filtered_signal = apply_median_filter(filtered_signal, kernel_size)
    # Step 3: Wavelet denoising
    denoised_signal = apply_wavelet_denoising(median_filtered_signal, wavelet, level)
    return denoised_signal

# Global dictionary to store feature names and values
feature_dict = {}

def add_feature(name, func, *args):
    """Try to add a feature by checking the shape and ensuring it’s a scalar."""
    try:
        value = func(*args)
        if np.isscalar(value) or (isinstance(value, np.ndarray) and value.size == 1):
            feature_dict[name] = value.item() if isinstance(value, np.ndarray) else value
        else:
            print(f"Warning: Feature '{name}' has incorrect shape and was not added.")
    except Exception as e:
        print(f"Error computing feature '{name}': {e}")

def extract_features(data, fs=100):
    features = []
    labels = []

    for key, signals in data.items():
        mod_type, snr = key
        for signal in signals:
            real_part, imag_part = signal[0], signal[1]
            # Apply preprocessing to real and imaginary parts
            real_part = preprocess_signal(real_part, fs)
            imag_part = preprocess_signal(imag_part, fs)
            complex_signal = real_part + 1j * imag_part

            # Reset feature dictionary for each signal
            global feature_dict
            feature_dict = {}

            # Add features with validation
            add_feature("Inst. Freq. Dev", instantaneous_frequency_deviation, complex_signal)
            add_feature("Spectral Entropy", spectral_entropy, real_part)
            add_feature("Envelope Mean", lambda x: envelope_mean_variance(x)[0], real_part)
            add_feature("Envelope Variance", lambda x: envelope_mean_variance(x)[1], real_part)
            add_feature("Spectral Flatness", spectral_flatness, real_part)
            add_feature("Spectral Peaks", lambda x: spectral_peaks_bandwidth(x)[0], real_part)
            add_feature("Bandwidth", lambda x: spectral_peaks_bandwidth(x)[1], real_part)
            add_feature("Zero Crossing Rate", zero_crossing_rate, real_part)
            add_feature("Amplitude Mean", lambda x: np.mean(compute_instantaneous_features(x)[0]), real_part)
            add_feature("Phase Variance", lambda x: np.var(compute_instantaneous_features(x)[1]), real_part)
            add_feature("Modulation Index", compute_modulation_index, real_part)
            add_feature("Spectral Sparsity", compute_spectral_asymmetry, real_part)
            add_feature("Envelope Ratio", lambda x: envelope_mean_variance(x)[0] / (envelope_mean_variance(x)[1] + 1e-10), real_part)
            add_feature("FFT Center Freq", lambda x: compute_fft_features(x)[0], real_part)
            add_feature("FFT Peak Power", lambda x: compute_fft_features(x)[1], real_part)
            add_feature("FFT Avg Power", lambda x: compute_fft_features(x)[2], real_part)
            add_feature("FFT Std Dev Power", lambda x: compute_fft_features(x)[3], real_part)
            add_feature("Kurtosis", compute_kurtosis, real_part)
            add_feature("Skewness", compute_skewness, real_part)
            add_feature("HOC-2", lambda x: compute_higher_order_cumulants(x, order=2), real_part)
            add_feature("HOC-3", lambda x: compute_higher_order_cumulants(x, order=3), real_part)
            add_feature("HOC-4", lambda x: compute_higher_order_cumulants(x, order=4), real_part)
            add_feature("Crest Factor", compute_crest_factor, real_part)
            add_feature("Spectral Entropy Value", compute_spectral_entropy, real_part)
            add_feature("Autocorr Decay", compute_autocorrelation_decay, real_part)
            add_feature("RMS Instant Freq", compute_rms_of_instantaneous_frequency, real_part)
            add_feature("Entropy Instant Freq", compute_entropy_of_instantaneous_frequency, real_part)
            add_feature("Envelope Variance", compute_envelope_variance, real_part)
            add_feature("PAPR", compute_papr, real_part)

            # Add SNR as a feature
            feature_dict["SNR"] = snr  # Include SNR as part of the features

            # Append the feature values and label
            features.append(list(feature_dict.values()))
            labels.append(mod_type)

    return np.array(features), labels

# Load and preprocess data
with open("../RML2016.10a_dict.pkl", "rb") as f:
    data = pickle.load(f, encoding="latin1")

features, labels = extract_features(data)

# Encode labels for classification
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Train/test split and classification
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.3, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model by SNR and plot results
unique_snrs = sorted(set(X_test[:, -1]))
accuracy_per_snr = []
for snr in unique_snrs:
    snr_indices = np.where(X_test[:, -1] == snr)
    X_snr = X_test[snr_indices]
    y_snr = y_test[snr_indices]
    y_pred = clf.predict(X_snr)
    accuracy = accuracy_score(y_snr, y_pred)
    accuracy_per_snr.append(accuracy * 100)
    print(f"SNR: {snr} dB, Accuracy: {accuracy * 100:.2f}%")

# Plot Recognition Accuracy vs. SNR
plt.figure(figsize=(10, 6))
plt.plot(unique_snrs, accuracy_per_snr, 'b-o', label='Recognition Accuracy')
plt.xlabel("SNR (dB)")
plt.ylabel("Recognition Accuracy (%)")
plt.title("Recognition Accuracy vs. SNR with Preprocessing")
plt.legend()
plt.grid(True)
plt.ylim(0, 100)
plt.show()

# Feature importance plot
feature_names = list(feature_dict.keys())
importances = clf.feature_importances_
plt.figure(figsize=(10, 8))
plt.barh(feature_names, importances, color='skyblue')
plt.xlabel("Feature Importance")
plt.title("Feature Importance for Modulation Classification with Preprocessing")
plt.show()

# Confusion matrix
y_pred_test = clf.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Modulation Classification with Preprocessing")
plt.show()

# Print Classification Report
print("Classification Report for Modulation Types with Preprocessing:")
print(classification_report(y_test, y_pred_test, target_names=label_encoder.classes_))
